# s03-Window Functions within SQLite

Window Functions get available sincle SQLite 3.25.According to the SQLite docs:

>A window function is a special SQL function where the input values are taken from a “window” of one or more rows in the results set of a SELECT statement.

SQLite now has these built-in Window functions:

- row_number()
- rank()
- dense_rank()
- percent_rank()
- cume_dist()
- ntile(N)
- lag(expr), lag(expr, offset), lag(expr, offset, default)
- lead(expr), lead(expr, offset), lead(expr, offset, default)
- first_value(expr)
- last_value(expr)
- nth_value(expr, N)

Previous tutorials are created based on a little bit older SQLite versions. At that time, window Functions are not available. In this tutoiral, I am going to touch some of them in the following 5 sections:

- Firstly, give a comparison between SQL Window Functions vs. SQL Aggregate Functions

- Secondly, go through a few WINDOW functions based on regular aggregate functions, such as **AVG**, **MIN/MAX**, **COUNT**, **SUM**.

- Thirdly, focus on rank-related functions, such as **ROW_NUMBER**, **RANK**,and **RANK_DENSE**. Those functions are handy when generating ranking indexes, and you need to be fluent in those functions before entering a data scientist SQL interview.

- Forthly, talk about generating statistics (e.g., percentiles, quartiles, median, etc .) with the **NTILE** function, a common task for a data scientist.

- Fifthly, focus on **LAG** and **LEAD**, two functions that are super important if you are interviewing for a role that requires dealing with time-series data.


## 1. [Window Functions vs. Aggregate Functions](https://learnsql.com/blog/window-functions-vs-aggregate-functions/)

In the previous tutorials, we presented some application cases of Aggregate Functions. What are the Similarities and Differences between them? Let's have a quick gothrough.

### 1.1 Aggregate functions

*Aggregate functions* operate on a set of values to return a single scalar value. These are SQL aggregate functions:

- AVG() returns the average of the specified values.
- SUM() calculates the sum of all values in the set.
- MAX() and MIN() return the maximum and minimum value, respectively.
-COUNT() returns the total number of values in the set.

By using the GROUP BY clause, you can calculate an aggregate value for several groups in one query. That is to say, ***aggregate functions*** collapse the individual rows and present the aggregate value for all the rows in the group.


### 1.2 Window functions

*Window functions* operate on a set of rows called a window frame. They return a single value for each row from the underlying query.

The window frame (or simply window) is defined using the OVER() clause. This clause also allows defining a window based on a specific column (similar to GROUP BY).

To calculate the returned values, window functions may use aggregate functions, but they will use them with the OVER() clause.

### 1.3 Similarities and Differences Between Window and Aggregate Functions

Now that we have seen both types of functions, we can summarize the similarities and differences between them.

***Both window functions and aggregate functions:***

- Operate on a set of values (rows).
- Can calculate aggregate amounts (e.g. AVG(), SUM(), MAX(), MIN(), or COUNT()) on the set.
- Can group or partition data on one or more columns.

***Aggregate functions with GROUP BY differ from window functions in that they:***

- Use GROUP BY() to define a set of rows for aggregation. 
- Group rows based on column values.
- Collapse rows based on the defined groups.


***Window functions differ from aggregate functions used with GROUP BY in that they:***

- Use OVER() instead of GROUP BY() to define a set of rows.
- May use many functions other than aggregates (e.g. RANK(), LAG(), or LEAD()).
- Groups rows on the row’s rank, percentile, etc. as well as its column value.
- Do not collapse rows.
- May use a sliding window frame (which depends on the current row).

## 2. Practive window functions

Let's connect the demo database, firstly.

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///data/demo.db3

### 2.1 WINDOW functions based on regular aggregate functions: AVG, MIN/MAX, SUM, COUNT

Window functions are functions that perform calculations across a set of rows related to the current row.

It is comparable to the type of calculation done with an aggregate function, but unlike regular aggregate functions, window functions do not group several rows into a single output row — the rows retain their own identities.

Behind the scenes, the window functions process more than just the query results' current row.

Let's take the ***watershed_monthly*** as the demo table.

#### 2.1.1 AVG

In [ ]:
%%sql sqlite://
SELECT
    YR,
    MO,
    PREC_mm,
    ROUND(AVG(PREC_mm) OVER(PARTITION BY MO)) AS avg_PREC_mm
FROM watershed_monthly
ORDER by MO

There is no **GROUP BY** clause for the AVG function, but how does the SQL engine know which rows to use to compute the average? The answer is the **PARTITION BY** clause inside the **OVER()** utility, and we are calculating the average based on a unique value of rating.

In the final output, every row has the average PREC_mm from the same month (*MO*). By adding a ***ORDER by***, you will find there is the same averge value for each month.

#### 2.1.2 SUM

Let’s take a look at a more complicated example, where we calculated a running sum with a window function.

In [ ]:
%%sql sqlite://
SELECT
    YR,
    MO,
    PREC_mm,    
    SUM(PREC_mm) OVER(ORDER BY MO) AS running_total,
    SUM(PREC_mm) OVER() AS overall,
    ROUND(SUM(PREC_mm) OVER(ORDER BY MO) * 100.0 /SUM(PREC_mm) OVER(), 2) AS running_percentage
FROM watershed_monthly

Notice that there is no **PARTITION BY** clause because I am not grouping those  into any PREC_mm sub-categories. I want to compute my overall progress but not based on any subgroups or categories.

Another thing to notice is that if I don’t add anything inside the **OVER()** function, I get the total number of PREC_mm from the entire table. As you can see from the second last column: they all have the same value of **24084.34**, but after I add the **ORDER BY** clause, I get the running total of the PREC_mm up to that specific row (**running_total** column).

#### 2.1.3 Others

You can try ***MIN***, ***COUNT**, ***AVG*** by yourself, which are all similar to prevois examples of ***SUM*** and ***MAX**.

### 2.2 ROW_NUMBER, RANK, DENSE_RANK


Let’s go through some of the essential WINDOW functions: **ROW_NUMBER** and **RANK**. In the following example, our goal is to create a ranking index based on the amount of PREC_mm for the entire table.

#### 2.2.1 ROW_NUMBER

In [ ]:
%%sql sqlite://
SELECT
    YR,
    MO,
    PREC_mm,  
    ROW_NUMBER() OVER(ORDER BY PREC_mm DESC) AS row_num
FROM watershed_monthly
ORDER by row_num

As you can see, the ROW_NUMBER function generates a sequence of integers, starting from 1, for each row.

#### 2.2.2 ROW_NUMBER over PARTITION

Let’s take a look at another example. Instead of comparing a amount of PREC_mm to all other PREC_mm from the entire table, we can rank them within each month using **PARTITION BY**.

In [ ]:
%%sql sqlite://
SELECT
    YR,
    MO,
    PREC_mm,  
    ROW_NUMBER() OVER(PARTITION BY MO ORDER BY PREC_mm DESC) AS row_num
FROM watershed_monthly
ORDER by row_num

**ROW_NUMBER** with **OVER** and **PARTITION BY** is a regular pattern that is frequently used in advanced SQL.

#### 2.2.3 Others

Now you can try the **RANK** and ***DENSE_RANK*** functions by yourself as they are very similar to **ROW_NUMBER**. The difference between **RANK** and **ROW_NUMBER** is that **RANK** assigns the same unique values if there is a tie and restarts the next value with the total number of rows up to that row.

**DENSE_RANK**. It is very similar to **RANK** but differs in how it handles ties. It restarts with the following immediate consecutive value rather than creating a gap.

### 2.3 NTILE

**NTILE** is a handy function, especially for data analytics professionals. For example, as a data scientist, you probably need to create robust statistics such as quartile, quintile, median, etc., in your daily job, and **NTILE** makes it very easy to generate those numbers.

**NTILE** takes an argument of the number of buckets and then creates this number of buckets as equally as possible, based on how the rows are partitioned and ordered inside the **OVER** function.

In [ ]:
%%sql sqlite://
SELECT
    YR,
    MO,
    PREC_mm,  
    NTILE(100) OVER(ORDER BY PREC_mm) AS percentile
FROM watershed_monthly
ORDER BY percentile;

You can create a few more statistics, such as **DECILES** (10 buckets) and **QUARTILES** (4 buckets).

### 2.4 LAG and Lead

**LAG** and **LEAD**, which are extremely useful for dealing with time-related data. The main difference between them is that LAG gets data from previous rows, while LEAD is the opposite, which fetches data from the following rows. We can use either one of the two functions to compare month-over-month growth. 

Their syntax is very similar to other window functions. Instead of focusing on the format of the syntax, let me show you a couple of examples.

#### 2.4.1 LAG

1. In the first step, we created a temporal table of ann_total with PREC_mm.

2. And in the second step, we appended the previous year’s Ann_PREC_mm to the current year’s using the LAG function.

3. Notice that last 2 columns of the first row are empty. It’s simply because 1981 is the first available year.

4. We also specified the offset, which is 1, so we fetch the next row. If you change this number to 2, then you compare the current year’s Ann_PREC_mm to the year before the previous year.

5. Finally, we divided the current year’s Ann_PREC_mm by the previous year’s to create our year Ann_PREC_mm growth.

In [ ]:
%%sql sqlite://
WITH annul_total AS (
    SELECT
        YR,    
        SUM(PREC_mm) AS Ann_PREC_mm    
    FROM watershed_monthly
    GROUP by YR
)

SELECT
    YR,
    Ann_PREC_mm,
    LAG(Ann_PREC_mm, 1) OVER (ORDER BY YR) prev_year_total,
    ROUND(Ann_PREC_mm *1.0/LAG(Ann_PREC_mm,1) OVER (ORDER BY YR), 2) AS dod
FROM annul_total
ORDER BY YR

#### 2.4.2 LEAD

Let’s take a look at another example. It’s very similar to the previous one, but instead of appending the previous year’s Ann_PREC_mm, we used the **LEAD** function with an offset of 1 to get the next year’s Ann_PREC_mm.

We then divided the next year’s Ann_PREC_mm by the current year’s Ann_PREC_mm to get the year-over-year growth.

Notice that last 2 columns of the last two rows are empty. It’s simply because 2010 and 2009 is the first two available years.

In [ ]:
%%sql sqlite://
WITH annul_total AS (
    SELECT
        YR,    
        SUM(PREC_mm) AS Ann_PREC_mm    
    FROM watershed_monthly
    GROUP by YR
)

SELECT
    YR,
    Ann_PREC_mm,
    LEAD(Ann_PREC_mm, 2) OVER (ORDER BY YR) prev_year_total,
    ROUND(Ann_PREC_mm *1.0/LEAD(Ann_PREC_mm,2) OVER (ORDER BY YR), 2) AS dod
FROM annul_total
ORDER BY YR

## Summary

Congratulations! If you have followed through all the examples and have seen most of the common **WINDOW functions/patterns**.

WINDOW functions are a family of SQL utilities, which provide some extremely powerful and useful features. But for many, since they are so foreign to standard SQL, they are difficult to learn and understand, have strange syntax—and are very often avoided. Writing a bug-free WINDOW function query could be quite challenging. It takes time and practice to become a master, and you are getting there soon.


## References and resources

https://sqlite.org/windowfunctions.html

https://learnsql.com/blog/window-functions-vs-aggregate-functions/

https://blog.jupyter.org/a-jupyter-kernel-for-sqlite-9549c5dcf551

https://blog.xojo.com/2018/12/18/sqlite-3-25-adds-window-functions-and-improves-alter-table/

https://sqlpad.io/playground/